In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
import xgboost as xgb
import lightgbm as lgb
import catboost as cb
import optuna
import warnings
warnings.filterwarnings('ignore')

In [2]:
train_df=pd.read_csv('/Users/karansoni/Downloads/train.csv')
test_df=pd.read_csv('/Users/karansoni/Downloads/test.csv')

In [3]:
train_df.head()

,time,A,B,C,D,E,F,G,H,I,J,K,L,M,N,Y1,Y2
0,0,0.207366,-0.159951,-0.634176,-0.580962,-0.266505,0.060173,-0.475257,-1.486516,-0.332594,-0.671466,-0.226149,-0.187624,-0.780237,-0.785965,-0.935902,-0.310081
1,1,0.188828,-0.265508,0.042143,-0.550442,-0.132319,-0.185219,0.028295,0.093210,-0.518139,-0.251917,-0.347845,-0.359069,-0.161254,0.020401,-0.089707,-0.305374
2,2,-0.144261,-0.577142,-0.214634,-0.747391,-0.184255,-0.464831,-0.085181,0.700449,-0.603438,0.197773,-0.566696,-0.580799,0.202726,0.135261,-0.077855,-0.631485
3,3,0.208982,-0.310449,0.513708,-0.562868,0.742308,-0.305487,0.762246,1.363020,-0.384575,0.525556,-0.348514,-0.428099,0.548993,0.471031,0.941271,-0.535212
4,4,0.093320,-0.358156,0.173188,-0.687296,-0.161461,-0.116062,-0.245748,0.863372,-0.655588,-0.263358,-0.557428,-0.481214,0.083602,0.003087,-0.039582,-0.490561


In [4]:
test_df.head()

,id,time,A,B,C,D,E,F,G,H,I,J,K,L,M,N
0,1,80005,-0.371888,-0.273485,0.538326,-0.377691,1.283159,-0.286957,0.439415,0.020831,-0.295480,0.117643,-0.369900,-0.378902,0.470166,0.411796
1,2,80006,-0.459598,-0.514915,-0.235153,-0.262379,0.044343,-0.429888,-0.059034,-0.725713,-0.407681,-0.255702,-0.461901,-0.249516,-0.216745,-0.328599
2,3,80007,-0.381609,-0.265023,-0.629131,-0.186288,-0.146302,-0.357875,0.114223,-0.359341,-0.190712,-0.404083,-0.272153,-0.193158,-0.679351,-0.476331
3,4,80008,-0.371423,-0.106279,-1.142702,-0.200429,-0.176918,-0.354048,-0.230199,-0.305226,0.185342,-0.333010,0.518678,-0.028775,-0.610620,-0.846772
4,5,80009,-0.309393,-0.015144,-1.099371,-0.405841,-0.239068,-0.403306,-0.983134,-1.067315,-0.105132,-0.809948,0.185197,-0.150254,-1.080190,-0.685936


In [5]:
train_df.isnull

<bound method DataFrame.isnull of         time         A         B         C         D         E         F  \
0          0  0.207366 -0.159951 -0.634176 -0.580962 -0.266505  0.060173   
1          1  0.188828 -0.265508  0.042143 -0.550442 -0.132319 -0.185219   
2          2 -0.144261 -0.577142 -0.214634 -0.747391 -0.184255 -0.464831   
3          3  0.208982 -0.310449  0.513708 -0.562868  0.742308 -0.305487   
4          4  0.093320 -0.358156  0.173188 -0.687296 -0.161461 -0.116062   
...      ...       ...       ...       ...       ...       ...       ...   
79995  80000 -0.339802 -0.372094 -0.120952 -0.363438  0.266299 -0.222866   
79996  80001 -0.421921 -0.222554 -0.689585 -0.512266 -0.206562 -0.494450   
79997  80002 -0.467038 -0.282018  0.010707 -0.589594  0.067635 -0.410820   
79998  80003 -0.369710 -0.322384 -0.540326 -0.563206 -0.568736 -0.550024   
79999  80004 -0.304899 -0.047375 -0.932102 -0.229424 -0.474211 -0.336000   

              G         H         I         J        

In [6]:
train_df.describe()

,time,A,B,C,D,E,F,G,H,I,J,K,L,M,N,Y1,Y2
count,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000
mean,40003.926125,0.068979,-0.004927,-0.000802,0.031913,-0.000292,0.063241,-0.000050,-0.002554,0.070158,-0.000028,-0.000803,0.018752,-0.000636,0.000849,-0.002807,-0.061172
std,23095.035045,1.078271,1.031053,1.010158,1.047274,1.036233,1.069182,1.036270,1.040708,1.064972,1.015265,1.038007,1.057857,1.012744,1.025396,0.970655,0.923702
min,0.000000,-0.642788,-0.771179,-30.068210,-0.899105,-29.530710,-0.737694,-35.715053,-23.598892,-0.762271,-34.806015,-0.729140,-0.734872,-29.342481,-49.869913,-28.918180,-0.850799
25%,20004.750000,-0.374171,-0.451919,-0.417830,-0.500604,-0.475700,-0.415164,-0.419804,-0.437469,-0.395450,-0.432185,-0.405350,-0.438401,-0.386474,-0.366045,-0.387523,-0.497802
50%,40004.500000,-0.264501,-0.285350,-0.003942,-0.279073,-0.006753,-0.236990,-0.004954,0.006673,-0.186564,0.003981,-0.263940,-0.292186,0.004210,0.005000,-0.001591,-0.318091
75%,60004.250000,0.005451,0.061020,0.414435,0.166928,0.458875,0.132404,0.421969,0.444982,0.160427,0.437449,0.033419,0.055187,0.387407,0.367437,0.392470,0.026905
max,80004.000000,24.917549,53.667013,29.312756,28.490356,39.322300,46.837612,22.023252,20.634973,33.604551,19.117766,49.278617,38.849874,27.179463,32.488227,29.859458,34.636039


In [7]:
X_train=train_df.drop(columns=['Y1','Y2'])

In [8]:
y1_train=train_df['Y1']
y2_train=train_df['Y2']
X_test=test_df.drop(columns=['id'])
test_ids=test_df['id']

In [36]:
def create_features(df, train=True, y1=None, y2=None):
    df = df.copy()
    features = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N']
    
    # Lag features 
    for feature in features:
        for lag in [1, 2, 3, 5]:
            df[f'{feature}_lag{lag}'] = df[feature].shift(lag)
    
    # Rolling statistics 
    for feature in features:
        for window in [3, 5, 10]:
            df[f'{feature}_roll_mean_{window}'] = df[feature].rolling(window=window).mean()
            df[f'{feature}_roll_std_{window}'] = df[feature].rolling(window=window).std()
            df[f'{feature}_roll_min_{window}'] = df[feature].rolling(window=window).min()
            df[f'{feature}_roll_max_{window}'] = df[feature].rolling(window=window).max()
    
    # interaction features 
    for i, f1 in enumerate(features[:5]):
        for f2 in features[i+1:5]:
            df[f'{f1}_{f2}_ratio'] = df[f1] / (df[f2] + 1e-6)
            df[f'{f1}_{f2}_product'] = df[f1] * df[f2]
    
    # timebased features
    df['time_diff'] = df['time'].diff().fillna(1)
    df['time_trend'] = df['time'] - df['time'].min()
    
    # Target encoding 
    if train:
        for feature in features:
            df[f'{feature}_y1_mean'] = df[feature].shift(1).map(y1.groupby(X_train[feature]).mean())
            df[f'{feature}_y2_mean'] = df[feature].shift(1).map(y2.groupby(X_train[feature]).mean())
    
    # Fill NaN values
    df = df.fillna(df.mean())
    return df

In [10]:
X_train=create_features(X_train,train=True,y1=y1_train,y2=y2_train)
X_test=create_features(X_test,train=False)

In [11]:
common_columns = [col for col in X_train.columns if col in X_test.columns]
X_train = X_train[common_columns]
X_test = X_test[common_columns]

In [12]:
scaler=StandardScaler()

In [13]:
X_train_scaled=scaler.fit_transform(X_train.drop(columns=['time']))
X_test_scaled=scaler.fit_transform(X_test.drop(columns=['time']))


In [14]:
feature_cols = [col for col in X_train.columns if col != 'time']
X_train_scaled = pd.DataFrame(X_train_scaled, columns=feature_cols)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=feature_cols)

In [15]:
def select_features(X, y, threshold='median'):
    model = xgb.XGBRegressor(random_state=42)
    model.fit(X, y)
    selector = SelectFromModel(model, threshold=threshold, prefit=True)
    selected_features = X.columns[selector.get_support()].tolist()
    return selected_features

In [16]:
from sklearn.feature_selection import SelectFromModel

In [17]:
selected_features_y1 = select_features(X_train_scaled, y1_train)
selected_features_y2 = select_features(X_train_scaled, y2_train)
selected_features = list(set(selected_features_y1 + selected_features_y2))

In [18]:
X_train_scaled = X_train_scaled[selected_features]
X_test_scaled = X_test_scaled[selected_features]

In [35]:
# Optuna objective function for XGBoost
def objective_xgb(trial, X, y):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.2, log=True),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'random_state': 42,
        'tree_method': 'hist',  
        'eval_metric': 'rmse' 
    
    model = xgb.XGBRegressor(**param)
    tscv = TimeSeriesSplit(n_splits=5)
    scores = []
    
    for train_idx, val_idx in tscv.split(X):
        X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]
        model.fit(X_tr, y_tr) 
        scores.append(r2_score(y_val, y_pred))
    
    return np.mean(scores)


def objective_lgb(trial, X, y):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.2, log=True),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 50),
        'random_state': 42,
        'metric': 'rmse' 
    }
    
    model = lgb.LGBMRegressor(**param)
    tscv = TimeSeriesSplit(n_splits=5)
    scores = []
    
    for train_idx, val_idx in tscv.split(X):
        X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]
        model.fit(
            X_tr, 
            y_tr, 
            eval_set=[(X_val, y_val)],
            eval_metric='rmse',
            callbacks=[lgb.early_stopping(stopping_rounds=10, verbose=False)]
        )
        y_pred = model.predict(X_val)
        scores.append(r2_score(y_val, y_pred))
    
    return np.mean(scores)


def objective_cb(trial, X, y):
    param = {
        'iterations': trial.suggest_int('iterations', 100, 1000),
        'depth': trial.suggest_int('depth', 4, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.2, log=True),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10),
        'random_seed': 42,
        'task_type': 'CPU'
    }
    
    model = cb.CatBoostRegressor(**param, verbose=False)
    tscv = TimeSeriesSplit(n_splits=5)
    scores = []
    
    for train_idx, val_idx in tscv.split(X):
        X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]
        model.fit(
            X_tr, 
            y_tr, 
            eval_set=(X_val, y_val),
            early_stopping_rounds=10,
            verbose=False
        )
        y_pred = model.predict(X_val)
        scores.append(r2_score(y_val, y_pred))
    
    return np.mean(scores)

In [20]:
study_xgb_y1 = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())
study_xgb_y1.optimize(lambda trial: objective_xgb(trial, X_train_scaled, y1_train), n_trials=10)

[I 2025-09-21 15:18:17,199] A new study created in memory with name: no-name-68af8a4d-9406-43a7-9854-d42e92df6fc0
[I 2025-09-21 15:23:34,625] Trial 0 finished with value: 0.7398831241136192 and parameters: {'n_estimators': 714, 'max_depth': 10, 'learning_rate': 0.00630021668318292, 'subsample': 0.7535719610025956, 'colsample_bytree': 0.8290885945002476, 'min_child_weight': 9, 'gamma': 1.6609941003126572}. Best is trial 0 with value: 0.7398831241136192.
[I 2025-09-21 15:24:15,102] Trial 1 finished with value: 0.7311233644748848 and parameters: {'n_estimators': 786, 'max_depth': 4, 'learning_rate': 0.11172277585782572, 'subsample': 0.7921743132876836, 'colsample_bytree': 0.8866481243731663, 'min_child_weight': 3, 'gamma': 3.554025494193403}. Best is trial 0 with value: 0.7398831241136192.
[I 2025-09-21 15:24:42,364] Trial 2 finished with value: 0.7363614813876289 and parameters: {'n_estimators': 337, 'max_depth': 6, 'learning_rate': 0.09365185447530003, 'subsample': 0.8783245733914226, '

In [21]:
study_lgb_y1 = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())
study_lgb_y1.optimize(lambda trial: objective_lgb(trial, X_train_scaled, y1_train), n_trials=10)


[I 2025-09-21 15:31:36,033] A new study created in memory with name: no-name-ec507b87-0223-4435-bfb7-d4034343d3f8


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.010812 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 75987
[LightGBM] [Info] Number of data points in the train set: 13335, number of used features: 298
[LightGBM] [Info] Start training from score 0.005932
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.017176 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 75990
[LightGBM] [Info] Number of data points in the train set: 26668, number of used features: 298
[LightGBM] [Info] Start training from score -0.001826
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.011847 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 75990
[LightGBM] [Info] Number of data points in the tra

[I 2025-09-21 15:32:10,367] Trial 0 finished with value: 0.6401440694976479 and parameters: {'n_estimators': 192, 'max_depth': 11, 'learning_rate': 0.005813601935190833, 'subsample': 0.7322503645703561, 'colsample_bytree': 0.7817816035376659, 'min_child_samples': 14}. Best is trial 0 with value: 0.6401440694976479.


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.009262 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 75987
[LightGBM] [Info] Number of data points in the train set: 13335, number of used features: 298
[LightGBM] [Info] Start training from score 0.005932
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.017451 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 75990
[LightGBM] [Info] Number of data points in the train set: 26668, number of used features: 298
[LightGBM] [Info] Start training from score -0.001826
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.025918 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 75990
[LightGBM] [Info] Number of data points in the train set: 40001, number of used features: 298
[LightGBM] [Info] St

[I 2025-09-21 15:32:24,940] Trial 1 finished with value: 0.7374584847599615 and parameters: {'n_estimators': 512, 'max_depth': 10, 'learning_rate': 0.07865504843964687, 'subsample': 0.6476741591794694, 'colsample_bytree': 0.7714647751725889, 'min_child_samples': 19}. Best is trial 1 with value: 0.7374584847599615.


...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Info] Auto-ch

[I 2025-09-21 15:35:02,030] Trial 8 finished with value: 0.7375813966440252 and parameters: {'n_estimators': 569, 'max_depth': 8, 'learning_rate': 0.011185179547756068, 'subsample': 0.906077639041128, 'colsample_bytree': 0.6335678240100497, 'min_child_samples': 8}. Best is trial 8 with value: 0.7375813966440252.


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.010937 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 75987
[LightGBM] [Info] Number of data points in the train set: 13335, number of used features: 298
[LightGBM] [Info] Start training from score 0.005932
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gai

[I 2025-09-21 15:35:19,300] Trial 9 finished with value: 0.7376445137743248 and parameters: {'n_estimators': 996, 'max_depth': 6, 'learning_rate': 0.061583785297038916, 'subsample': 0.8635289812015767, 'colsample_bytree': 0.679780674339373, 'min_child_samples': 21}. Best is trial 9 with value: 0.7376445137743248.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


In [22]:
study_cb_y1 = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())
study_cb_y1.optimize(lambda trial: objective_cb(trial, X_train_scaled, y1_train), n_trials=10)

[I 2025-09-21 15:35:19,308] A new study created in memory with name: no-name-e9247349-9492-4b9d-a2c0-7a33d67263fe
[I 2025-09-21 15:36:50,960] Trial 0 finished with value: 0.7313463967766969 and parameters: {'iterations': 284, 'depth': 7, 'learning_rate': 0.054006773641040146, 'l2_leaf_reg': 8.920571437714779}. Best is trial 0 with value: 0.7313463967766969.
[I 2025-09-21 15:41:20,901] Trial 1 finished with value: 0.6917228847272112 and parameters: {'iterations': 116, 'depth': 10, 'learning_rate': 0.020219813893255004, 'l2_leaf_reg': 4.566082423656094}. Best is trial 0 with value: 0.7313463967766969.
[I 2025-09-21 15:55:39,673] Trial 2 finished with value: 0.7325141834826197 and parameters: {'iterations': 918, 'depth': 10, 'learning_rate': 0.024815783386107237, 'l2_leaf_reg': 2.513989251353321}. Best is trial 2 with value: 0.7325141834826197.
[I 2025-09-21 15:55:56,390] Trial 3 finished with value: 0.7289501234184351 and parameters: {'iterations': 900, 'depth': 5, 'learning_rate': 0.084

In [23]:
study_xgb_y2 = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())
study_xgb_y2.optimize(lambda trial: objective_xgb(trial, X_train_scaled, y2_train), n_trials=10)
study_lgb_y2 = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())
study_lgb_y2.optimize(lambda trial: objective_lgb(trial, X_train_scaled, y2_train), n_trials=10)
study_cb_y2 = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())
study_cb_y2.optimize(lambda trial: objective_cb(trial, X_train_scaled, y2_train), n_trials=10)

[I 2025-09-21 16:22:22,275] A new study created in memory with name: no-name-ad82abff-78e9-4ee5-9ac8-1c5389347980
[I 2025-09-21 16:22:40,092] Trial 0 finished with value: 0.6536517938246273 and parameters: {'n_estimators': 123, 'max_depth': 8, 'learning_rate': 0.10413501544836042, 'subsample': 0.9090963513999027, 'colsample_bytree': 0.660718282135503, 'min_child_weight': 2, 'gamma': 3.7492914657382026}. Best is trial 0 with value: 0.6536517938246273.
[I 2025-09-21 16:25:35,459] Trial 1 finished with value: 0.6603537787231291 and parameters: {'n_estimators': 602, 'max_depth': 7, 'learning_rate': 0.0054979276602213295, 'subsample': 0.7091130620364853, 'colsample_bytree': 0.7070344060380627, 'min_child_weight': 9, 'gamma': 1.3429027689001327}. Best is trial 1 with value: 0.6603537787231291.
[I 2025-09-21 16:26:02,611] Trial 2 finished with value: 0.6314715808596247 and parameters: {'n_estimators': 319, 'max_depth': 3, 'learning_rate': 0.010641250172572064, 'subsample': 0.9100974427907883,

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.008725 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 75987
[LightGBM] [Info] Number of data points in the train set: 13335, number of used features: 298
[LightGBM] [Info] Start training from score 0.082652
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.018395 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 75990
[LightGBM] [Info] Number of data points in the train set: 26668, number of used features: 298
[LightGBM] [Info] Start training from score -0.102749
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.027033 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 75990
[LightGBM] [Info] Number of data points in the train set: 40001, number of used features: 298
[LightGBM] [Info] St

[I 2025-09-21 16:32:05,485] Trial 0 finished with value: 0.6783432484824481 and parameters: {'n_estimators': 960, 'max_depth': 9, 'learning_rate': 0.08570960473378032, 'subsample': 0.7467446711809546, 'colsample_bytree': 0.6489380319029744, 'min_child_samples': 19}. Best is trial 0 with value: 0.6783432484824481.


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.009912 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 75987
[LightGBM] [Info] Number of data points in the train set: 13335, number of used features: 298
[LightGBM] [Info] Start training from score 0.082652
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gai

[I 2025-09-21 16:32:37,028] Trial 1 finished with value: 0.6557090172183289 and parameters: {'n_estimators': 782, 'max_depth': 3, 'learning_rate': 0.0073695729263393754, 'subsample': 0.6400487344621515, 'colsample_bytree': 0.608267247577983, 'min_child_samples': 15}. Best is trial 0 with value: 0.6783432484824481.


...

[I 2025-09-21 16:36:39,615] Trial 7 finished with value: 0.670798680560199 and parameters: {'n_estimators': 366, 'max_depth': 5, 'learning_rate': 0.07473264844244441, 'subsample': 0.6177666590371796, 'colsample_bytree': 0.6540804241359709, 'min_child_samples': 19}. Best is trial 0 with value: 0.6783432484824481.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.009749 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 75987
[LightGBM] [Info] Number of data points in the train set: 13335, number of used features: 298
[LightGBM] [Info] Start training from score 0.082652
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.018050 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 75990
[LightGBM

[I 2025-09-21 16:37:29,013] Trial 8 finished with value: 0.6659976976940719 and parameters: {'n_estimators': 364, 'max_depth': 12, 'learning_rate': 0.012311868142171379, 'subsample': 0.6323442370173522, 'colsample_bytree': 0.8041467525503674, 'min_child_samples': 46}. Best is trial 0 with value: 0.6783432484824481.


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.009165 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 75987
[LightGBM] [Info] Number of data points in the train set: 13335, number of used features: 298
[LightGBM] [Info] Start training from score 0.082652
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.020577 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 75990
[LightGBM] [Info] Number of data points in the train set: 26668, number of used features: 298
[LightGBM] [Info] Start training from score -0.102749
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.025615 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 75990
[LightGBM] [Info] Number of data points in the train set: 40001, number of used features: 298
[LightGBM] [Info] St

[I 2025-09-21 16:37:39,077] Trial 9 finished with value: 0.6698885246753837 and parameters: {'n_estimators': 605, 'max_depth': 7, 'learning_rate': 0.11482277274260436, 'subsample': 0.9732658277193345, 'colsample_bytree': 0.7451441498904445, 'min_child_samples': 6}. Best is trial 0 with value: 0.6783432484824481.
[I 2025-09-21 16:37:39,079] A new study created in memory with name: no-name-064bd768-be05-422c-a42b-096960be5571
[I 2025-09-21 16:40:21,932] Trial 0 finished with value: 0.6639302952270473 and parameters: {'iterations': 945, 'depth': 10, 'learning_rate': 0.15795273318964223, 'l2_leaf_reg': 4.828400089535805}. Best is trial 0 with value: 0.6639302952270473.
[I 2025-09-21 16:40:34,945] Trial 1 finished with value: 0.628869022486366 and parameters: {'iterations': 123, 'depth': 4, 'learning_rate': 0.02614443975289756, 'l2_leaf_reg': 3.1100276539012}. Best is trial 0 with value: 0.6639302952270473.
[I 2025-09-21 16:40:50,786] Trial 2 finished with value: 0.6744228932441751 and para

In [24]:
xgb_y1 = xgb.XGBRegressor(**study_xgb_y1.best_params)
lgb_y1 = lgb.LGBMRegressor(**study_lgb_y1.best_params)
cb_y1 = cb.CatBoostRegressor(**study_cb_y1.best_params, verbose=False)
xgb_y2 = xgb.XGBRegressor(**study_xgb_y2.best_params)
lgb_y2 = lgb.LGBMRegressor(**study_lgb_y2.best_params)
cb_y2 = cb.CatBoostRegressor(**study_cb_y2.best_params, verbose=False)

In [37]:
tscv = TimeSeriesSplit(n_splits=5)
y1_oof_preds = np.zeros(len(y1_train))
y2_oof_preds = np.zeros(len(y2_train))
y1_test_preds = np.zeros((len(X_test_scaled), 3))  # 3 models
y2_test_preds = np.zeros((len(X_test_scaled), 3))

In [26]:
model_weights_y1 = {'xgb': [], 'lgb': [], 'cb': []}
model_weights_y2 = {'xgb': [], 'lgb': [], 'cb': []}

In [66]:
for i, (train_idx, val_idx) in enumerate(tscv.split(X_train_scaled)):
    X_tr, X_val = X_train_scaled.iloc[train_idx], X_train_scaled.iloc[val_idx]
    y1_tr, y1_val = y1_train.iloc[train_idx], y1_train.iloc[val_idx]
    y2_tr, y2_val = y2_train.iloc[train_idx], y2_train.iloc[val_idx]
    
    # Train and predict for Y1
    xgb_y1.fit(X_tr, y1_tr)
    lgb_y1.fit(X_tr, y1_tr)
    cb_y1.fit(X_tr, y1_tr)
    y1_oof_preds[val_idx] = (xgb_y1.predict(X_val) + lgb_y1.predict(X_val) + cb_y1.predict(X_val)) / 3
    y1_test_preds[:, 0] += xgb_y1.predict(X_test_scaled) / 5
    y1_test_preds[:, 1] += lgb_y1.predict(X_test_scaled) / 5
    y1_test_preds[:, 2] += cb_y1.predict(X_test_scaled) / 5
    
    # Train and predict for Y2
    xgb_y2.fit(X_tr, y2_tr)
    lgb_y2.fit(X_tr, y2_tr)
    cb_y2.fit(X_tr, y2_tr)
    y2_oof_preds[val_idx] = (xgb_y2.predict(X_val) + lgb_y2.predict(X_val) + cb_y2.predict(X_val)) / 3
    y2_test_preds[:, 0] += xgb_y2.predict(X_test_scaled) / 5
    y2_test_preds[:, 1] += lgb_y2.predict(X_test_scaled) / 5
    y2_test_preds[:, 2] += cb_y2.predict(X_test_scaled) / 5

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005383 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 48450
[LightGBM] [Info] Number of data points in the train set: 13335, number of used features: 190
[LightGBM] [Info] Start training from score 0.005932
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gai

In [28]:
meta_model_y1 = LinearRegression()
meta_model_y2 = LinearRegression()

In [29]:
meta_features_y1 = np.column_stack([
    xgb_y1.predict(X_train_scaled),
    lgb_y1.fit(X_train_scaled, y1_train).predict(X_train_scaled),
    cb_y1.fit(X_train_scaled, y1_train).predict(X_train_scaled)
])
meta_features_y2 = np.column_stack([
    xgb_y2.predict(X_train_scaled),
    lgb_y2.fit(X_train_scaled, y2_train).predict(X_train_scaled),
    cb_y2.fit(X_train_scaled, y2_train).predict(X_train_scaled)
])

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.034795 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 75990
[LightGBM] [Info] Number of data points in the train set: 80000, number of used features: 298
[LightGBM] [Info] Start training from score -0.002807
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive ga

In [30]:
meta_model_y1.fit(meta_features_y1, y1_train)
meta_model_y2.fit(meta_features_y2, y2_train)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [31]:
y1_pred = meta_model_y1.predict(y1_test_preds)
y2_pred = meta_model_y2.predict(y2_test_preds)

In [32]:
submission = pd.DataFrame({
    'id': test_ids,
    'Y1': y1_pred,
    'Y2': y2_pred
})

In [33]:
submission.to_csv('submission.csv', index=False)
print("Submission file 'submission.csv' created successfully!")

Submission file 'submission.csv' created successfully!


In [34]:
print(f"Y1 Validation R²: {r2_score(y1_train, y1_oof_preds)}")
print(f"Y2 Validation R²: {r2_score(y2_train, y2_oof_preds)}")

Y1 Validation R²: 0.643513675125795
Y2 Validation R²: 0.5476583455818638
